<a href="https://colab.research.google.com/github/EthanW67/Machine-Learning-Research/blob/main/Research_Projects/Revised_UNet_6_27_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using the resized 28x28 to modified UNet architecture.
* Resized Original 28x28 NMIST dataset to 7x7 then resized 7x7 → 28x28
* Compared and trained resized 28x28 with the original 28x28 NMIST dataset.


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.transforms import Resize
from torchmetrics.image import StructuralSimilarityIndexMeasure, PeakSignalNoiseRatio
device = "cuda" if torch.cuda.is_available() else "cpu"
psnr = PeakSignalNoiseRatio()
psnr.to(device)
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim.to(device)
import matplotlib.pyplot as plt




## Load Dataset

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformations
original_transforms = transforms.Compose([
    transforms.ToTensor(), # Convert to tensor
    transforms.Resize((28, 28)),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image
])

transform_down = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Resize((7, 7)),  # Resize to 7x7
])

transform_up = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize back to 28x28
    transforms.Normalize((0.5,), (0.5,))  # Normalize the image

])

# Regular Dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=original_transforms)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=original_transforms)

# Load MNIST dataset and apply the downscale transformation
train_dataset_down = datasets.MNIST(root='./data', train=True, download=True, transform=transform_down)
test_dataset_down = datasets.MNIST(root='./data', train=False, download=True, transform=transform_down)

# Apply the upscale transformation on the already downscaled datasets
train_dataset_up = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transform_down, transform_up]))
test_dataset_up = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([transform_down, transform_up]))

batch_size = 32

# Original Dataset
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Create DataLoader instances for the downscaled version
train_loader_down = DataLoader(dataset=train_dataset_down, batch_size=batch_size, shuffle=False)
test_loader_down = DataLoader(dataset=test_dataset_down, batch_size=batch_size, shuffle=False)

# Create DataLoader instances for the upscaled version from downscaled images
train_loader_up = DataLoader(dataset=train_dataset_up, batch_size=batch_size, shuffle=False)
test_loader_up = DataLoader(dataset=test_dataset_up, batch_size=batch_size, shuffle=False)


# Get the dimensions of the original dataset

for batch_idx, (data, target) in enumerate(train_loader):
    print(data.size())  # Should print torch.Size([64, 1, 28, 28])
    break
# Example of how you might check the sizes to confirm the process
for batch_idx, (data, target) in enumerate(train_loader_down):
    print(f"Size of batch {batch_idx + 1} in train_loader_down: {data.size()}")  # 7x7 images
    break
for batch_idx, (data, target) in enumerate(train_loader_up):
    print(f"Size of batch {batch_idx + 1} in train_loader_up: {data.size()}")  # 28x28 images
    break

## Function to plot images


In [ ]:
def plot_images(images, titles):
    fig, axes = plt.subplots(1, len(images), figsize=(10, 3))
    for ax, img, title in zip(axes, images, titles):
        ax.imshow(img.squeeze(), cmap='gray')
        ax.set_title(title)
        ax.axis('off')
    plt.show()

# Load a single batch from each DataLoader
original_data, _ = next(iter(train_loader))
downscaled_data, _ = next(iter(train_loader_down))
upscaled_data, _ = next(iter(train_loader_up))

# Visualize the first image in the batch
plot_images([original_data[0], downscaled_data[0], upscaled_data[0]], ['Original', 'Downscaled', 'Upscaled'])

# Calculate and print PSNR and SSIM
original_img = original_data[0].unsqueeze(0).to(device)
upscaled_img = upscaled_data[0].unsqueeze(0).to(device)

psnr_score = psnr(upscaled_img, original_img).item()
ssim_score = ssim(upscaled_img, original_img).item()
print(f"PSNR Score: {psnr_score}")
print(f"SSIM Score: {ssim_score}")

## Create U-Net Model


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class u_net(nn.Module):
    def __init__(self, dropout_rate = 0.5):
        super(u_net, self).__init__()
        # Encoder
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout2d(p=dropout_rate)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout2d(p=dropout_rate)

        # Bottleneck
        self.bottleneck = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.bottleneck2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        # Decoder
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout2d(p=dropout_rate)

        self.upconv2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.conv7 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.conv8 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.dropout4 = nn.Dropout2d(p=dropout_rate)

        # Output Layer
        self.outconv = nn.Conv2d(32, 1, kernel_size=3, padding=1)

        # ReLU Layer
        self.relu = nn.ReLU()

    def forward(self, x):
        # Encode
        x = self.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.relu(self.conv2(x))
        x = self.dropout1(x)
        skip_connection1 = x
        x = self.pool(x)

        x = self.relu(self.conv3(x))
        x = self.bn2(x)
        x = self.relu(self.conv4(x))
        x = self.dropout2(x)
        skip_connection2 = x
        x = self.pool(x)

        # Bottleneck
        x = self.relu(self.bottleneck(x))
        x = self.bn3(x)
        x = self.relu(self.bottleneck2(x))
        # Decoder
        x = self.upconv1(x)
        x = self.relu(self.conv5(torch.cat((skip_connection2, x), 1))) # Concatenate after upsampling
        x = self.bn4(x)
        x = self.relu(self.conv6(x))
        x = self.dropout3(x)

        x = self.upconv2(x)
        x = self.relu(self.conv7(torch.cat((skip_connection1, x), 1))) # Concatenate after upsampling
        x = self.bn5(x)
        x = self.relu(self.conv8(x))
        x = self.dropout4(x)

        # Output Layer
        x = self.outconv(x)
        return x

## Train Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = u_net().to(device)
#criterion = torch.nn.L1Loss()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)



epoch_numbers = []
psnr_values = []
ssim_values = []

# Define a function to train the model
def train(model, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for (images, _), (images_up, _) in zip(train_loader, train_loader_up):
            # Move tensors to the appropriate device
            images = images.to(device)
            images_up = images_up.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward + backward + optimize
            outputs = model(images_up)
            loss = criterion(outputs, images)  # Using the original images as both input and target
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}: Loss {running_loss / len(train_loader)}')

        PSNR_value, SSIM_value = PSNR_SSIM_Value(model, train_loader, train_loader_up)

        epoch_numbers.append(epoch + 1)
        psnr_values.append(PSNR_value)
        ssim_values.append(SSIM_value)

# Train the model
#train(model, criterion, optimizer, num_epochs = 2)

## Useful Functions

In [ ]:
def validate(model, loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():  # No need to track gradients
        for images, _ in loader:
            images = images.to(device)
            high_res_images = images.to(device)

            outputs = model(images)
            loss = criterion(outputs, high_res_images)
            total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f'Validation Loss: {avg_loss}')
    return avg_loss


def PSNR_SSIM_Value(model, loader, loader_up):
    model.eval()
    with torch.inference_mode():
        for (images, _), (images_up, _) in zip(loader, loader_up):
            images = images.to(device)
            images_up = images_up.to(device)
            #print(images.shape)
            outputs = model(images_up)
            #print(outputs.shape)
            PSNR_value = psnr(outputs, images)
            SSIM_value = ssim(outputs, images)
            print(PSNR_value)
            print(SSIM_value)
            break
    return  PSNR_value.item(), SSIM_value.item()


## Visualize

In [ ]:
def visualize_results_UNet(model, loader, loader_down, loader_up, criterion, optimizer, num_epochs):
    train(model, criterion, optimizer, num_epochs)

    # Plot the epoch numbers vs. PSNR and SSIM
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epoch_numbers, psnr_values, label='PSNR')
    print(epoch_numbers)
    print(psnr_values)
    plt.xlabel('Epoch Number')
    plt.ylabel('PSNR')
    plt.title('Epoch Number vs. PSNR')
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epoch_numbers, ssim_values, label='SSIM')
    print(ssim_values)
    plt.xlabel('Epoch Number')
    plt.ylabel('SSIM')
    plt.title('Epoch Number vs. SSIM')
    plt.grid(True)

    plt.show()

    model.eval()
    with torch.no_grad():
        for (images, _), (images_down, _), (images_up, _) in zip(loader, loader_down, loader_up):

            images = images.to(device)
            images_down = images_down.to(device)
            images_up = images_up.to(device)
            output = model(images_up)

            print(images.shape)
            print(images_down.shape)
            print(images_up.shape)
            print(output.shape)
            # print(transform_down)

            # Convert first 4 images in the batch from PyTorch tensors to NumPy arrays
            original_images = images[:5].cpu().numpy()
            seven_images = images_down[:5].cpu().numpy()
            twenty_eight_images = images_up[:5].cpu().numpy()
            reconstructed_images = output[:5].cpu().numpy()

            # Plotting
            fig, axes = plt.subplots(4, 5, figsize=(12, 9))
            for i in range(5):
                ax = axes[0][i]
                ax.imshow(original_images[i][0], cmap='gray', interpolation='none')
                ax.title.set_text('Original Image')
                ax.axis('off')

                ax = axes[1][i]
                ax.imshow(seven_images[i][0], cmap='gray', interpolation='none')
                ax.title.set_text('7x7 Images')
                ax.axis('off')

                ax = axes[2][i]
                ax.imshow(twenty_eight_images[i][0], cmap='gray', interpolation='none')
                ax.title.set_text('28x28 Images')
                ax.axis('off')

                ax = axes[3][i]
                ax.imshow(reconstructed_images[i][0], cmap='gray', interpolation='none')
                ax.title.set_text('Result Images')
                ax.axis('off')

            plt.show()
            break  # Only show one batch of images

visualize_results_UNet(model, test_loader, test_loader_down, test_loader_up, criterion, optimizer, num_epochs=5)

